In [157]:
import numpy as np
import pandas as pd
from geopy.distance import distance as geodist
from tqdm import tqdm
import itertools as itr
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('.data/datathon_2024_dataset_corrected.csv')
df["date"] = pd.to_datetime(df["game_date"], format='%Y%m%d')
print(df.head())

   Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
0           0   20000329       NYN       CHN        False           3   
1           1   20000330       CHN       NYN        False           1   
2           2   20000403       ATL       COL         True           2   
3           3   20000403       CIN       MIL         True           3   
4           4   20000403       FLO       SFN        False           6   

   away_score  venue        venue_name        city  ... home_pa  home_1b  \
0           5  TOK01        Tokyo Dome       Tokyo  ...      38        5   
1           5  TOK01        Tokyo Dome       Tokyo  ...      44        5   
2           0  ATL02      Turner Field     Atlanta  ...      32        5   
3           3  CIN08     Cinergy Field  Cincinnati  ...      20        3   
4           4  MIA01  Sun Life Stadium       Miami  ...      38        9   

   home_2b  home_3b  home_hr  home_fo  home_so  home_bb  home_hbp       date  
0        1        0      

In [3]:
df = df[['date', 'home_team', 'away_team', 'is_day_game', 'home_score', 'away_score', 'venue_name']]
print(df.head())

        date home_team away_team  is_day_game  home_score  away_score  \
0 2000-03-29       NYN       CHN        False           3           5   
1 2000-03-30       CHN       NYN        False           1           5   
2 2000-04-03       ATL       COL         True           2           0   
3 2000-04-03       CIN       MIL         True           3           3   
4 2000-04-03       FLO       SFN        False           6           4   

         venue_name  
0        Tokyo Dome  
1        Tokyo Dome  
2      Turner Field  
3     Cinergy Field  
4  Sun Life Stadium  


In [129]:
venue_df = pd.read_csv('.data/venue.csv', index_col='venue_name')
print(venue_df.head())

                     lat     long
venue_name                       
Tokyo Dome        35.705  139.752
Turner Field      33.735  -84.389
Cinergy Field     39.548  -84.303
Sun Life Stadium  25.959  -80.240
Stade Olympique   45.558  -73.552


In [4]:
years = df['date'].dt.year.unique()
years.sort()
print(years)

[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]


In [5]:
teams = df['home_team'].unique()
teams.sort()
print(teams)

['ANA' 'ARI' 'ATL' 'BAL' 'BOS' 'CHA' 'CHN' 'CIN' 'CLE' 'COL' 'DET' 'FLO'
 'HOU' 'KCA' 'LAN' 'MIA' 'MIL' 'MIN' 'MON' 'NYA' 'NYN' 'OAK' 'PHI' 'PIT'
 'SDN' 'SEA' 'SFN' 'SLN' 'TBA' 'TEX' 'TOR' 'WAS']


In [6]:
runs_score = pd.DataFrame(data=0, index=years, columns=teams)
runs_allow = pd.DataFrame(data=0, index=years, columns=teams)
n_games = pd.DataFrame(data=0, index=years, columns=teams)

In [7]:
for ind in tqdm(df.index):
    home_team = df.loc[ind, 'home_team']
    away_team = df.loc[ind, 'away_team']
    home_runs = df.loc[ind, 'home_score']
    away_runs = df.loc[ind, 'away_score']
    yr = df.loc[ind, 'date'].year

    runs_score.loc[yr, home_team] += home_runs
    runs_score.loc[yr, away_team] += away_runs
    
    runs_allow.loc[yr, home_team] += away_runs
    runs_allow.loc[yr, away_team] += home_runs

    n_games.loc[yr, home_team] += 1
    n_games.loc[yr, away_team] += 1

100%|█| 56775/56775 [


In [8]:
# Pythagenpat
exponent = ((runs_score + runs_allow) / n_games) ** 0.287
baseline_winrate = 1 / (1 + (runs_allow / runs_score) ** exponent)
print(baseline_winrate)

           ANA       ARI       ATL       BAL       BOS       CHA       CHN  \
2000  0.497152  0.523468  0.559723  0.431799  0.529139  0.576102  0.418589   
2001  0.474424  0.588558  0.557681  0.411695  0.516936  0.501815  0.548383   
2002  0.628879  0.591116  0.600708  0.431408  0.619417  0.534116  0.466007   
2003  0.495536  0.521192  0.597707  0.452974  0.584673  0.547743  0.527076   
2004  0.562105  0.327163  0.585816  0.507012  0.602689  0.519659  0.579561   
2005  0.577651  0.402326  0.561438  0.455864  0.560040  0.563878  0.492768   
2006  0.521478  0.490796  0.525893  0.423728  0.497044  0.543347  0.427591   
2007  0.555880  0.487027  0.547541  0.433470  0.628956  0.409914  0.540199   
2008  0.543646  0.509163  0.484448  0.448743  0.592822  0.550595  0.613511   
2009  0.571788  0.460949  0.562879  0.419721  0.581185  0.494841  0.523493   
2010  0.485953  0.424522  0.573157  0.387204  0.545301  0.530928  0.447154   
2011  0.523760  0.545834  0.525928  0.407792  0.582219  0.464834

In [9]:
opening_day = {yr : df['date'][df['date'].dt.year == yr].min() for yr in years}

df['day'] = 0
for ind in tqdm(df.index):
    df.loc[ind, 'day'] = (df.loc[ind, 'date'] - opening_day[df.loc[ind, 'date'].year]).days

print(df.head())

100%|█| 56775/56775 [

        date home_team away_team  is_day_game  home_score  away_score  \
0 2000-03-29       NYN       CHN        False           3           5   
1 2000-03-30       CHN       NYN        False           1           5   
2 2000-04-03       ATL       COL         True           2           0   
3 2000-04-03       CIN       MIL         True           3           3   
4 2000-04-03       FLO       SFN        False           6           4   

         venue_name  day  
0        Tokyo Dome    0  
1        Tokyo Dome    1  
2      Turner Field    5  
3     Cinergy Field    5  
4  Sun Life Stadium    5  


In [95]:
homes = pd.DataFrame(data="", index=years, columns=teams)
for ind in tqdm(df.index):
    home_team = df.loc[ind, 'home_team']
    yr = df.loc[ind, 'date'].year
    venue = df.loc[ind, 'venue_name']
    homes.loc[yr, home_team] = venue
print(homes)

100%|█| 56775/56775 [

                           ANA          ARI           ATL  \
2000  Angel Stadium of Anaheim  Chase Field  Turner Field   
2001  Angel Stadium of Anaheim  Chase Field  Turner Field   
2002  Angel Stadium of Anaheim  Chase Field  Turner Field   
2003  Angel Stadium of Anaheim  Chase Field  Turner Field   
2004  Angel Stadium of Anaheim  Chase Field  Turner Field   
2005  Angel Stadium of Anaheim  Chase Field  Turner Field   
2006  Angel Stadium of Anaheim  Chase Field  Turner Field   
2007  Angel Stadium of Anaheim  Chase Field  Turner Field   
2008  Angel Stadium of Anaheim  Chase Field  Turner Field   
2009  Angel Stadium of Anaheim  Chase Field  Turner Field   
2010  Angel Stadium of Anaheim  Chase Field  Turner Field   
2011  Angel Stadium of Anaheim  Chase Field  Turner Field   
2012  Angel Stadium of Anaheim  Chase Field  Turner Field   
2013  Angel Stadium of Anaheim  Chase Field  Turner Field   
2014  Angel Stadium of Anaheim  Chase Field  Turner Field   
2015  Angel Stadium of A

In [115]:
days = list(np.arange(0, df['day'].max()+1, 1))
print(days)

extended_days = [-1] + list(days) + [days[-1]+1]
print(extended_days)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 

In [149]:
location = {team:
                {year:
                    {day:
                         {'home': False,
                          'away': False,
                          'travel': False,
                          'miles': 0,
                          'venue': None,
                          'won_game': False,
                          'games': 0,
                          'wins': 0,
                          'wab': baseline_winrate.loc[year, team],
                          'dwab': 0}
                     for day in extended_days
                    }
                for year in years
                }
            for team in teams
            }

for team in teams:
    for yr in years:
        home_venue = homes.loc[yr, team]
        if home_venue is None:
            print(team, yr)
            del location[team][yr]
        else:
            location[team][yr][-1]['venue'] = home_venue
            location[team][yr][days[-1]+1]['venue'] = home_venue

In [150]:
for ind in tqdm(df.index):
    home_team = df.loc[ind, 'home_team']
    away_team = df.loc[ind, 'away_team']
    home_score = df.loc[ind, 'home_score']
    away_score = df.loc[ind, 'away_score']
    yr = df.loc[ind, 'date'].year
    day = df.loc[ind, 'day']
    venue = df.loc[ind, 'venue_name']
    location[home_team][yr][day]['home'] = True
    location[away_team][yr][day]['away'] = True    
    location[home_team][yr][day]['venue'] = venue
    location[away_team][yr][day]['venue'] = venue
    if home_score != away_score:
        if home_score > away_score:
            location[home_team][yr][day]['won_game'] = True
        else:
            location[away_team][yr][day]['won_game'] = True

100%|█| 56775/56775 [


In [151]:
for team in teams:
    for yr in years:
        for day in reversed(days):
            if location[team][yr][day]['venue'] is None:
                location[team][yr][day]['venue'] = location[team][yr][day+1]['venue']
        for day in extended_days[1:]:
            yesterday_venue = location[team][yr][day-1]['venue']
            today_venue = location[team][yr][day]['venue']
            if yesterday_venue != today_venue:
                location[team][yr][day]['travel'] = True
                location[team][yr][day]['miles'] = geodist(venue_df.loc[yesterday_venue].tolist(), venue_df.loc[today_venue].tolist()).miles
            location[team][yr][day]['wins'] = location[team][yr][day-1]['wins'] + location[team][yr][day]['won_game']
            location[team][yr][day]['games'] = location[team][yr][day-1]['games'] + location[team][yr][day]['home'] + location[team][yr][day]['away']
            if location[team][yr][day]['games'] > 0:
                location[team][yr][day]['wab'] = location[team][yr][day]['wins'] - baseline_winrate.loc[yr, team] * location[team][yr][day]['games']
                location[team][yr][day]['dwab'] = location[team][yr][day]['wab'] - location[team][yr][day-1]['wab']
        if location[team][yr][days[-1]+1]['games'] == 0:
            del location[team][yr]

In [163]:
var_names = ['win', 'baseline_odds'] + [f'{var}{delay}' for var, delay in itr.product(['home', 'away', 'travel', 'miles'], range(21))]
print(var_names)
ml_df = pd.DataFrame(data=np.nan, columns=var_names, index=[0])

['win', 'baseline_odds', 'home0', 'home1', 'home2', 'home3', 'home4', 'home5', 'home6', 'home7', 'home8', 'home9', 'home10', 'home11', 'home12', 'home13', 'home14', 'home15', 'home16', 'home17', 'home18', 'home19', 'home20', 'away0', 'away1', 'away2', 'away3', 'away4', 'away5', 'away6', 'away7', 'away8', 'away9', 'away10', 'away11', 'away12', 'away13', 'away14', 'away15', 'away16', 'away17', 'away18', 'away19', 'away20', 'travel0', 'travel1', 'travel2', 'travel3', 'travel4', 'travel5', 'travel6', 'travel7', 'travel8', 'travel9', 'travel10', 'travel11', 'travel12', 'travel13', 'travel14', 'travel15', 'travel16', 'travel17', 'travel18', 'travel19', 'travel20', 'miles0', 'miles1', 'miles2', 'miles3', 'miles4', 'miles5', 'miles6', 'miles7', 'miles8', 'miles9', 'miles10', 'miles11', 'miles12', 'miles13', 'miles14', 'miles15', 'miles16', 'miles17', 'miles18', 'miles19', 'miles20']


In [ ]:
ind = 0
for team in teams:
    for yr in location[team].keys():
        for day in location[team][yr].keys():
            ml_df.loc[ind, 'win'] = 